# 언어 모델링 (Language Modeling)

* Input : 하나의 문장
* Output : Input 문장에 대한 확률
* 텍스트 분류 문제와 달리, 비지도 학습이다.
* 하지만, 순서가 있는 지도학습으로 문제를 바꿔서 풀 수 있다.

# 자기회귀 언어 모델링 (Autoregressive Language Modeling)
* Sequence가 주어졌을 때 이 문장에게 점수를 부여하는 방법이다.
* 이전 토큰이 나왔을 때 다음 토큰이 나올 확률을 계산하는 작업과 동일하다.
* 이렇게 정의하면서 비지도학습 문제를 지도학습으로 푸는 문제로 변하게 된다.
- 즉, 이전에 공부했던 텍스트 분류 문제와 같아지는데, Input은 이전에 나온 토큰, Output은 다음에 나올 토큰을 예측하는 것이다.
- 또한, 문장에 점수(Scroe)를 부여하는 방식은 이간이 말하는 사고 방식과 동일하다.
- 점수는 음의 최대 우도 (Negative Log Likelihood)로 측정한다.

# 인공신경망이 나오기전의 언어 모델 : N-Gram
- n-gram이란 주어진 텍스트 샘플의 연속적인 n개의 항목을 뜻한다.


- n-gram probability
  - 주어진 N개의 토큰에 태하여, 토큰 $x$가 나올 수 있는 확률은 이전 N개의 토큰과 토큰 $x$의 결합확률을 이전 N개의 토큰들과 등장할 수 있는 모든 토큰들의 결합확률의 합으로 나눈 것이다.
  - $p(x|x_{-N},x_{-N+1},...,x_{-1}) = {p(x_{-N},x_{-N+1},...,x_{-1},x) \over p(x|x_{-N},x_{-N+1},...,x_{-1})} = {p(x_{-N},x_{-N+1},...,x_{-1},x) \over \sum_{x \in V} p(x|x_{-N},x_{-N+1},...,x_{-1})}$


- n-gram probability의 최대우도추정(Maximum Likelihood Estimation)과정을 거친다. 확률을 추정하는 과정을 등장횟수로 대체하여 계산한다.
  - 1. 어떤 n-gram($x_{-N},x_{-N+1},...,x_{-1},x$)의 등장횟수를 센다. 이를 $c(x_{-N},x_{-N+1},...,x_{-1},x)$라고 하자.
  - 2. 주어진 n-gram에서 나올 수 잇는 모든 경우의 수 $(x_{-N},x_{-N+1},...,x_{-1},?)$의 등장횟수를 센다. 
  - 3. 따라서 등장확률은 $p(x|x_{-N},x_{-N+1},...,x_{-1}) = {p(x_{-N},x_{-N+1},...,x_{-1},x) \over \sum_{x' \in V} p(x|x_{-N},x_{-N+1},...,x_{-1},x')}$
  

- 하지만, 이 방법은 두 가지 문제점이 존재한다.
  - 1. 데이터 희소성(Data Sparsity) 문제 : 데이터가 희소할 경우 일반화가 어렵다.
       - 처음 보는 단어가 등장하면, 이전의 데이터에 존재하지 않았기 때문에 확률이 0이 된다.
  - 2. 장기의존성(Long-Term dependencies)을 캡쳐할 수 없다.
       - n개의 토큰만 조회하기 때문에, 긴 문장에 대한 다음 토큰은 추론이 불가능 하다.

- 장기의존성 문제를 해결하기 위해 조금 더 떨어진 토큰 간의 상관 관계를 보려면 n을 늘려야 하는데, 그러면 데이터의 희소성 문제가 발생한다.


- 해결방법
  - 1. 데이터 희소성 문제 : 셀 수 없기 때문에 문제가 된다.
       - Smoothing : 작은 상수 항을 더해준다.
       - Back off : Count가 0이 나올 시에 측정하는 n을 하나씩 줄여서 다시 Count한다.
  - 2. 장기의존성 문제
       - n-gram probability 방식으로는 해결할 수 없다.
       
       
- 직접 구현하지 않고, KenLM 패키지를 사용하는 것이 좋다.

# N-Gram 예시
 - "New York" 다음에 "University"가 나올 확률은?
 - 먼저 전체 Text에서 "New York University"가 나온 수를 센다.
 - 그 다음, "New York"으로 시작하는 세 자리 단어를 다 센다. (ex. New York city, New York State...)
 - 그 중에서"New York University"의 개수를 센다.
    - 하지만, 만약 "New York University"가 처음 나오는 단어라면...확률이 0이 되어버린다. : 데이터 희소성 문제
    - 지금은 n이 3이지만, n이 늘어나면 해결이 불가하다..... : 장기의존성 문제

# Neural N-Gram Language Model
 - 신경망을 사용함으로써 데이터 희소성 문제를 해결할 수 있다.
 - 기존의 카운트 기반 모델 보다 훈련 데이터에서 나오지 않았었던 N-Gram을 계산할 수가 있다.
 - 어떻게 이것이 가능할까? 데이터 희소성 문제가 생기는 이유를 살펴보자.
   - 간단한 대답은 토큰들이 훈련시에는 새기지 않지만, 테스트시에만 생기기 때문이다.
   - 조금 더 깊은 대답은 이산 공간(discrete space)에서 카운트하여 토큰들의 유사도 측정이 불가능하기 때문이다.
   - 하지만, 신경망에서는 토큰을 연속 벡터 공간(contiunous vector space)에 매핑(mapping)시킨다. 나오지 않았던 단어들도 유사도를 게산을 통해 연속 벡터 공간에서 의미가 있는 분포를 찾을 수 있게 된다. 이러한 학습을 통해 데이터 희소성 문제를 해결할 수 있다.

# 장기의존성(Long Term Dependency) 문제
 - 신경망을 이용한 n-gram 모델이 장기의존성 문제를 해결하지 못한 이유는 단순히 n-gram을 보기 때문이다.
 - 신경망을 사용하여 데이터 희소성 문제는 없어졌지만, 단순하게 n을 늘리게 되면 네트워크 규모가 커져서 학습해야할 매개변수가 많아지고, 이에 따른 데이터도 많이 필요하게 된다.
 
 
#### 해결방법 1. CNN(Convolution Language Model)
 - Dilated convolution을 사용하면 커버할 수 있는 토큰의 범위가 넓어진다.
 - 다만 텍스트 분류 때 처럼 쓰기에는 문제가 있다. 현재까지의 토큰을 보고 미래의 토큰을 예측하는 것인데, 아무 제약 없이 사용하면, 미래의 토큰을 보고 미래의 토큰을 예측해버리게 된다. 이는 더 이상 언어 모델링이 아니다.
 - 따라서, 매번 예측할 때 전에 나왔던 토큰들만 사용하도록, 미래의 토큰을 mask out 시킨다.

- Casual sentence representation
    - Convolution Language Model에서 보았듯이, 미래의 토큰을 보지않고 예측하기 위해 mask out하는 과정을 거쳤다.
    - 미래의 토큰을 보지 않고 다음 토큰을 예측하는 과정을 인과관계에 따른 문장표현(Casual sentence representation)이라고 한다.
    - 이는 인과관계에 따른 문장표현만 지킨다면 문맥을 무한대로 확장가능하다고 볼 수 있다.


#### 해결방법 2. CBoW
 - CBoW방법이 잘 안되는 이유는 토큰들 간의 순서를 무시하기 때문이다. 언어 모델링에서 문장은 의미 뿐만 아니라 문장의 구조도 중요하다.


#### 해결방법 3. Recurrent Language Model
 - 장점 : 실시간 처리(시간에 따른 문장 처리)
 - 단점 : 모든 문맥 내용을 하나의 고정된 벡터에 압축시켜서 학습시켜야 한다.
 - 응용 : 자동완성(autocomplete), 키워드 추천(Keyword suggestion), scoring partial hypotheses in generation
 
 
#### 해결방법 4. Recurrent Memory Networks
 - RNN과 Self Attention 방법을 결합하여 Recurrent Language Model의 단점을 보완한 방법이다.